In [ ]:
import os, json, time
from typing import TypedDict, Optional, Dict, Any, List

import requests
from openai import OpenAI
from langgraph.graph import StateGraph, START, END
from google.colab import userdata

# ---------------------------
# 1. OpenAI client
# ---------------------------

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

def chat_completion(messages, model="gpt-4o-mini", temperature=0.2) -> str:
    """Makes a chat completion API call to OpenAI."""
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return resp.choices[0].message.content

# ============================
# 2. Tools: Live data (Wikipedia, weather, FX)
# ============================

def fetch_city_info(city: str) -> str:
    """
    Fetch plain-text summary for the city from Wikipedia REST API.
    Used as live "RAG" context.
    """
    city_enc = city.replace(" ", "_")
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{city_enc}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Educational AI Demo - IEEE Workshop)",
        "Accept": "application/json"
    }
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print(f"[RAG] WARNING: could not fetch Wikipedia summary for {city}: {resp.text}")
        return ""
    data = resp.json()
    extract = data.get("extract", "")
    if not extract:
        print(f"[RAG] WARNING: no extract found in Wikipedia response for {city}")
        return ""
    return extract

def get_weather(city: str) -> Dict[str, Any]:
    """Fetches live weather information for a given city using Open-Meteo."""
    geo_url = "https://geocoding-api.open-meteo.com/v1/search"
    geo_params = {"name": city, "count": 1}
    geo_resp = requests.get(geo_url, params=geo_params).json()
    if "results" not in geo_resp or not geo_resp["results"]:
        return {"error": f"Could not find coordinates for city: {city}"}
    lat = geo_resp["results"][0]["latitude"]
    lon = geo_resp["results"][0]["longitude"]

    weather_url = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,temperature_2m_min",
        "forecast_days": 5,
        "timezone": "auto",
    }
    forecast = requests.get(weather_url, params=weather_params).json()
    return forecast

def convert_currency(amount: float, frm: str = "INR", to: str = "JPY") -> Dict[str, Any]:
    """Converts a given amount from one currency to another using Frankfurter API."""
    url = "https://api.frankfurter.app/latest"
    params = {"amount": amount, "from": frm, "to": to}
    resp = requests.get(url, params=params).json()
    return resp

TOOL_LOG: List[Dict[str, Any]] = []

def log_tool_call(tool_name, params, result, status="SUCCESS"):
    """Logs the details of a tool call, including its parameters, result, and status."""
    TOOL_LOG.append({
        "tool": tool_name,
        "params": params,
        "status": status,
        "result_preview": str(result)[:200],
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()),
    })

def safe_call_tool(tool_name: str, **kwargs) -> Any:
    """A governance layer for calling tools, including an allow-list and logging."""
    allowed = {
        "get_weather": get_weather,
        "convert_currency": convert_currency,
    }
    if tool_name not in allowed:
        raise ValueError(f"Tool '{tool_name}' not allowed by policy.")
    try:
        result = allowed[tool_name](**kwargs)
        log_tool_call(tool_name, kwargs, result, "SUCCESS")
    except Exception as e:
        result = {"error": str(e)}
        log_tool_call(tool_name, kwargs, result, "ERROR")
    return result

# ============================
# 3. Agents: planner, reviewer
# ============================

def planner_agent(user_query: str) -> str:
    """Generates a plan of subtasks based on the user's query."""
    prompt = f"""
You are a planner agent for a travel assistant.

User request:
{user_query}

Break this into 3–6 clear subtasks like:
- Understand constraints
- Research city
- Check weather and budget in local currency
- Propose day-wise plan
- Provide safety tips

Return a simple numbered list (no JSON).
"""
    return chat_completion(
        [{"role": "user", "content": prompt}],
        model="gpt-4o-mini",
        temperature=0.2,
    )

def reviewer_agent(answer: str, city_context: str, tool_context: str) -> Dict[str, Any]:
    """
    Reviews an generated answer for consistency, hallucinations, and safety.
    Optionally rewrites the answer if issues are found.
    """
    review_prompt = f"""
You are an AI governance reviewer.

You will:
1. Check if the Answer is broadly consistent with CityContext and ToolContext.
2. Flag obvious hallucinations (made-up facts, impossible weather, fake currency behavior).
3. Ensure the tone is safe, non-discriminatory, and suitable for all users.
4. If needed, rewrite the answer to be safer and more honest.

Return JSON:
{{
  "verdict": "APPROVE" or "REWRITE",
  "issues": ["..."],
  "revised_answer": "..."
}}

CityContext:
{city_context}

ToolContext:
{tool_context}

Answer:
{answer}
"""
    raw = chat_completion(
        [{"role": "user", "content": review_prompt}],
        model="gpt-4o-mini",
        temperature=0.1,
    )
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        data = {
            "verdict": "APPROVE",
            "issues": ["Reviewer missed JSON format; default APPROVE."],
            "revised_answer": ""
        }
    return data

# ============================
# 4. LangGraph State Definition
# ============================

class TravelState(TypedDict, total=False):
    user_query: str
    city: str
    trip_days: int
    budget_in_inr: float

    plan: str
    city_context: str
    city_summary: str
    tool_context: str
    raw_answer: str
    final_answer: str
    review: Dict[str, Any]

# ============================
# 5. LangGraph Node Functions
# ============================

def plan_node(state: TravelState) -> TravelState:
    """LangGraph node for generating a plan using the planner agent."""
    uq = state["user_query"]
    print("\n[plan_node] Running planner_agent...")
    plan = planner_agent(uq)
    print("[plan_node] Plan:\n", plan)
    return {"plan": plan}

def rag_node(state: TravelState) -> TravelState:
    """LangGraph node for fetching city information from Wikipedia and summarizing it."""
    city = state["city"]
    days = state["trip_days"]
    print("\n[rag_node] Fetching city context for:", city)
    ctx = fetch_city_info(city)
    if not ctx:
        summary = "I don't have enough reliable information from live sources."
    else:
        prompt = f"""
You are a careful travel expert.

Use ONLY the context below. Do NOT invent historical facts.
If something is not supported by the context, say:
"I don't know based on the provided information."

Context about {city}:
{ctx}

Task: Briefly describe this city for a tourist planning a {days}-day trip.
"""
        summary = chat_completion(
            [{"role": "user", "content": prompt}],
            model="gpt-4o-mini",
            temperature=0.2,
        )
    print("[rag_node] City summary generated.")
    return {"city_context": ctx, "city_summary": summary}

def tools_node(state: TravelState) -> TravelState:
    """LangGraph node for calling live weather and currency conversion tools."""
    city = state["city"]
    budget = state["budget_in_inr"]
    print("\n[tools_node] Calling live tools for weather + FX...")
    weather = safe_call_tool("get_weather", city=city)
    fx = safe_call_tool("convert_currency", amount=budget, frm="INR", to="JPY")

    parts = []
    if isinstance(weather, dict) and "daily" in weather and "temperature_2m_max" in weather["daily"]:
        dates = weather["daily"]["time"]
        tmin = weather["daily"]["temperature_2m_min"]
        tmax = weather["daily"]["temperature_2m_max"]
        lines = []
        for d, lo, hi in zip(dates, tmin, tmax):
            lines.append(f"{d}: min {lo}°C, max {hi}°C")
        parts.append("Weather forecast (next 5 days):\n" + "\n".join(lines))
    else:
        parts.append(f"Weather info error/format issue: {weather}")

    parts.append(f"Currency conversion result: {fx}")
    tool_ctx = "\n\n".join(parts)
    print("[tools_node] Tools executed.")
    return {"tool_context": tool_ctx}

def synthesize_node(state: TravelState) -> TravelState:
    """LangGraph node for synthesizing an itinerary based on plan, RAG, and tool context."""
    print("\n[synthesize_node] Generating initial itinerary...")
    user_query = state["user_query"]
    plan = state.get("plan", "")
    city_summary = state.get("city_summary", "")
    tool_ctx = state.get("tool_context", "")
    city = state["city"]
    days = state["trip_days"]
    budget = state["budget_in_inr"]

    synthesis_prompt = f"""
You are an AI travel planner.

User request:
{user_query}

Planning breakdown:
{plan}

City knowledge (from live web):
{city_summary}

Live tools (weather + currency):
{tool_ctx}

Now create a {days}-day travel plan with:
- Short overview (city + weather + season hints)
- Day-by-day itinerary
- Budget estimate (ranges) in INR and JPY (use FX info, but mention "approximate")
- 4–6 safety & cultural tips
- If anything is uncertain, clearly say "approximate" or "I am unsure".

Do NOT invent exact real-time prices. Use ranges and clearly mark them as estimates.
"""
    raw_answer = chat_completion(
        [{"role": "user", "content": synthesis_prompt}],
        model="gpt-4o-mini",
        temperature=0.4,
    )
    print("[synthesize_node] Initial itinerary generated.")
    return {"raw_answer": raw_answer}

def review_node(state: TravelState) -> TravelState:
    """LangGraph node for running the governance review on the generated answer."""
    print("\n[review_node] Running governance review...")
    raw_answer = state.get("raw_answer", "")
    city_context = state.get("city_context", "")
    tool_context = state.get("tool_context", "")

    review = reviewer_agent(raw_answer, city_context, tool_context)
    verdict = review.get("verdict", "APPROVE")
    if verdict == "REWRITE" and review.get("revised_answer"):
        final_answer = review["revised_answer"]
    else:
        final_answer = raw_answer
    print("[review_node] Review verdict:", verdict)
    return {"review": review, "final_answer": final_answer}

# ============================
# 6. Build LangGraph Graph
# ============================

builder = StateGraph(TravelState)

# Add nodes
builder.add_node("plan", plan_node)
builder.add_node("rag", rag_node)
builder.add_node("tools", tools_node)
builder.add_node("synthesize", synthesize_node)
builder.add_node("review", review_node)

# Add edges
builder.add_edge(START, "plan")
builder.add_edge("plan", "rag")
builder.add_edge("rag", "tools")
builder.add_edge("tools", "synthesize")
builder.add_edge("synthesize", "review")
builder.add_edge("review", END)

# Compile graph
travel_graph = builder.compile()

print("LangGraph travel graph built successfully.")

LangGraph travel graph built successfully.


In [ ]:
# Define initial state
initial_state: TravelState = {
    "user_query": "Plan a family-friendly 5-day trip from Delhi to Tokyo in January within a budget of ₹1,00,000.",
    "city": "Tokyo",
    "trip_days": 5,
    "budget_in_inr": 100000,
}

# Option A: simple single-shot run
result_state = travel_graph.invoke(initial_state)

print("\n================= FINAL GOVERNED ANSWER (LangGraph) =================\n")
print(result_state["final_answer"])

print("\n================= GOVERNANCE REVIEW =================\n")
print(json.dumps(result_state["review"], indent=2))

print("\n================= TOOL LOG =================\n")
for entry in TOOL_LOG:
    print(entry)



[plan_node] Running planner_agent...
[plan_node] Plan:
 1. Understand constraints: Confirm the number of family members traveling, specific interests (cultural, adventure, etc.), and any dietary or accessibility needs.

2. Research city: Explore family-friendly attractions in Tokyo, including parks, museums, and entertainment options suitable for all ages.

3. Check weather and budget in local currency: Look up the average weather in Tokyo for January and convert the budget of ₹1,00,000 into Japanese Yen (JPY) to ensure it covers flights, accommodation, food, and activities.

4. Propose day-wise plan: Create a detailed itinerary for each day, including suggested activities, dining options, and travel logistics.

5. Provide safety tips: Compile essential safety information, including emergency contacts, health precautions, and general travel advice for families in Tokyo.

[rag_node] Fetching city context for: Tokyo
[rag_node] City summary generated.

[tools_node] Calling live tools for

In [ ]:
import gradio as gr
import copy

def travel_ui_langgraph(city: str, trip_days: int, budget_in_inr: float, from_city: str = "Delhi"):
    """
    Streaming UI wrapper around the LangGraph-based travel_graph.
    It uses travel_graph.stream(...) to show what each node is doing.
    """
    # reset tool log for each run
    global TOOL_LOG
    TOOL_LOG = []

    log_text = ""

    def log_step(msg: str):
        """Appends a message to the log text and returns the updated log."""
        nonlocal log_text
        log_text += msg + "\n"
        return log_text

    # 1) Build initial user_query + initial state
    user_query = (
        f"Plan a family-friendly {trip_days}-day trip from {from_city} to {city} "
        f"within a budget of ₹{int(budget_in_inr):,}."
    )

    state: TravelState = {
        "user_query": user_query,
        "city": city,
        "trip_days": int(trip_days),
        "budget_in_inr": float(budget_in_inr),
    }

    # Initial log
    log = log_step(f"👋 Received request: {user_query}")
    yield log, ""  # only logs at first, no answer yet

    # 2) Stream through the LangGraph execution
    log = log_step("\n🚦 Starting LangGraph workflow: plan → rag → tools → synthesize → review")
    yield log, ""

    # travel_graph.stream returns a generator of events
    for event in travel_graph.stream(state):
        # event is typically a dict: {node_name: {updated_state...}}
        for node_name, node_update in event.items():
            # LangGraph may use special keys like "__start__", "__end__"
            if node_name in ("__start__", "__end__"):
                continue

            # Update our local state
            if isinstance(node_update, dict):
                state.update(node_update)

            # Build a nice message per node
            updated_keys = list(node_update.keys()) if isinstance(node_update, dict) else []
            log = log_step(f"\n🔹 Node '{node_name}' executed. Updated: {updated_keys}")

            # Show previews for important nodes
            if "plan" in updated_keys:
                log = log_step("📋 Planner output (short preview):\n" + state["plan"][:300] + ("..." if len(state["plan"]) > 300 else ""))
            if "city_summary" in updated_keys:
                log = log_step("🌐 City summary (from live Wikipedia, preview):\n" + state["city_summary"][:300] + ("..." if len(state["city_summary"]) > 300 else ""))
            if "tool_context" in updated_keys:
                log = log_step("🛠 Tool context (weather + FX, preview):\n" + state["tool_context"][:300] + ("..." if len(state["tool_context"]) > 300 else ""))
            if "raw_answer" in updated_keys:
                log = log_step("📝 Initial itinerary generated (before governance).")
            if "review" in updated_keys:
                review_json = json.dumps(state["review"], indent=2)
                log = log_step("🛡 Governance review:\n" + review_json[:700] + ("..." if len(review_json) > 700 else ""))

            # While the graph is running, we can already show partial answer if available
            partial_answer = state.get("raw_answer", "")
            yield log, partial_answer

    # 3) Final answer from state (governed)
    final_answer = state.get("final_answer") or state.get("raw_answer", "No answer generated.")
    log = log_step("\n✅ Workflow completed. Final answer ready.")
    log = log_step("\n📚 Tool log entries: " + str(len(TOOL_LOG)))

    yield log, final_answer


with gr.Blocks() as demo:
    gr.Markdown(
        """
    # 🌍 Agentic AI Travel Planner (LangGraph) — Live Demo

    This demo shows an **end-to-end Agentic AI application** built with **LangGraph**:
    - Planner node
    - Live web knowledge (Wikipedia)
    - Live tools (weather + FX)
    - Governance / hallucination checks
    """
    )

    with gr.Row():
        # Left column: inputs
        with gr.Column(scale=1):
            gr.Markdown("### ✏️ Input")
            city_in = gr.Textbox(label="Destination City", value="Tokyo")
            days_in = gr.Slider(label="Number of Days", minimum=1, maximum=14, step=1, value=5)
            budget_in = gr.Number(label="Budget in INR", value=100000)
            from_city_in = gr.Textbox(label="From City", value="Delhi")
            run_btn = gr.Button("🚀 Run LangGraph Agent")

        # Middle column: live log
        with gr.Column(scale=2):
            gr.Markdown("### 📡 Live Execution Log (LangGraph Nodes)")
            log_box = gr.Textbox(
                label="What the graph is doing (plan, rag, tools, synthesize, review)...",
                lines=26,
                interactive=False,
                show_copy_button=True,
            )

        # Right column: final answer
        with gr.Column(scale=2):
            gr.Markdown("### ✅ Final Itinerary & Tips (Governed)")
            output_box = gr.Textbox(
                label="Final governed response",
                lines=26,
                interactive=False,
                show_copy_button=True,
            )

    run_btn.click(
        fn=travel_ui_langgraph,
        inputs=[city_in, days_in, budget_in, from_city_in],
        outputs=[log_box, output_box]
    )

demo.queue().launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e5c1067035e4d585f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
